#  项目背景：
在互联网的时代下，电商平台提供给网民很多便利，如：提升了购物选择性、更直观的展示出各类商品的优惠折扣以及降低了购买成本等等。网购已经逐渐渗透进我们的生活。淘宝是电商圈里龙头企业，创造过无数奇迹，如“双十一”购物热潮及单日交易额百亿元等历史性的突破。淘宝平台不停的更新迭代，提供了更多的个性化服务。淘宝在2003年创立的，2012年注册会员近5亿，日活跃用户超1.2亿。接下来会通过淘宝2014年11月18日至2014年12月18日的随机用户行为的数据对淘宝进行用户行为分析，找出问题并提出优化方案。


# 数据说明：
数据来源：https://tianchi.aliyun.com/dataset/dataDetail?dataId=72423

本数据集共有104万条左右数据，数据为淘宝APP2014年11月18日至2014年12月18日的用户行为数据，共计6列字段。
* user_id：用户身份，脱敏
* item_id：商品ID，脱敏
* behavior_type：用户行为类型（包含点击、收藏、加购物车、支付四种行为，分别用数字1、2、3、4表示）
* user_geohash：地理位置
* item_category：品类ID（商品所属的品类）
* time：用户行为发生的时间

# 搭建业务指标




![Image Name](https://cdn.kesci.com/upload/image/qnzmibnj69.png)





# 数据分析过程
## 数据描述

导入工具包

In [3]:
import numpy as np
import pandas as pd
from pyecharts.charts import *
import pyecharts.options as opts
from pyecharts.components import Table
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.options import ComponentTitleOpts
import warnings
#忽略警告
warnings.filterwarnings('ignore')

读取数据

In [6]:
data = pd.read_csv('./淘宝用户行为.csv')
data.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11
3,98662432,320593836,1,96nn52n,6562,2014-12-06 10
4,98145908,290208520,1,NaN,13926,2014-12-16 21


查看数据基本信息

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   item_id        int64 
 2   behavior_type  int64 
 3   user_geohash   object
 4   item_category  int64 
 5   time           object
dtypes: int64(4), object(2)
memory usage: 561.1+ MB


 数据清洗

数据缺失值处理

In [8]:
data.isnull().sum()

user_id                0
item_id                0
behavior_type          0
user_geohash     8334824
item_category          0
time                   0
dtype: int64

异常值处理

In [9]:
data.describe()

,user_id,item_id,behavior_type,item_category
count,1.225691e+07,1.225691e+07,1.225691e+07,1.225691e+07
mean,7.170732e+07,2.023084e+08,1.105271e+00,6.846162e+03
std,4.122920e+07,1.167397e+08,4.572662e-01,3.809922e+03
min,4.913000e+03,6.400000e+01,1.000000e+00,2.000000e+00
25%,3.584965e+07,1.014130e+08,1.000000e+00,3.721000e+03
50%,7.292804e+07,2.021359e+08,1.000000e+00,6.209000e+03
75%,1.073774e+08,3.035405e+08,1.000000e+00,1.029000e+04
max,1.424559e+08,4.045625e+08,4.000000e+00,1.408000e+04


In [10]:
#'''当输入include=['O']，会计算离散型变量的统计特征'''
data.describe(include=['O'])

,user_geohash,time
count,3922082,12256906
unique,575458,744
top,94ek6ke,2014-12-11 22
freq,1052,54797


去重

对data去重，发现有多行完全重复，但基于时间column单位是小时： 同一用户可以在一小时内对商品多次收藏多次取消，甚至多次购买，故不去重

In [11]:
# 缺失值集中在用户地理信息列，故删去
del data['user_geohash']

一致化处理

* 将时间拆分为日期，小时

In [12]:
data['date'] = data['time'].apply(lambda x:x.split(' ')[0])
data['hour'] = data['time'].apply(lambda x:x.split(' ')[1])

* 将time、date调整为datetime，hour调整为int64

In [13]:
data['time']=pd.to_datetime(data['time'])
data['date']=pd.to_datetime(data['date'])
data['hour']=data['hour'].astype('int64')

* 将behavior_type调整为对应的行为，{1:pv,	2:fav,	3:cart	,4:buy}

In [14]:
data.loc[data['behavior_type']==1,'behavior_type']='pv'
data.loc[data['behavior_type']==2,'behavior_type']='fav'
data.loc[data['behavior_type']==3,'behavior_type']='cart'
data.loc[data['behavior_type']==4,'behavior_type']='buy'

* 确认时间在给定范围内

In [15]:
data[(data['date']>'2014-12-18') | (data['date']<'2014-11-18')]

,user_id,item_id,behavior_type,item_category,time,date,hour


重新查看数据分布

In [16]:
data.describe()

,user_id,item_id,item_category,hour
count,1.225691e+07,1.225691e+07,1.225691e+07,1.225691e+07
mean,7.170732e+07,2.023084e+08,6.846162e+03,1.481799e+01
std,4.122920e+07,1.167397e+08,3.809922e+03,6.474778e+00
min,4.913000e+03,6.400000e+01,2.000000e+00,0.000000e+00
25%,3.584965e+07,1.014130e+08,3.721000e+03,1.000000e+01
50%,7.292804e+07,2.021359e+08,6.209000e+03,1.600000e+01
75%,1.073774e+08,3.035405e+08,1.029000e+04,2.000000e+01
max,1.424559e+08,4.045625e+08,1.408000e+04,2.300000e+01


In [17]:
#'''当输入include=['O']，会计算离散型变量的统计特征'''
data.describe(include=['O'])

,behavior_type
count,12256906
unique,4
top,pv
freq,11550581


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   behavior_type  object        
 3   item_category  int64         
 4   time           datetime64[ns]
 5   date           datetime64[ns]
 6   hour           int64         
dtypes: datetime64[ns](2), int64(4), object(1)
memory usage: 654.6+ MB


通过观察数据集无异常值存在。

### 双十一宏观指标

In [19]:
# PV：总浏览页面数
total_pv = data["user_id"].count()
# UV：独立访客数
total_uv = data["user_id"].nunique()
# OD：总订单数
total_od = data[data['behavior_type']=="buy"].behavior_type.count()

In [20]:
table = Table()

headers = ["指标名称",'指标数']
rows = [
    ["PV：总浏览页面人数",total_pv],
    ["UV：独立访客数",total_uv],
    ["OD：总订单数",total_od],
    ["OD/PV：浏览到购买转化率", "{:.2f}%".format((total_od/total_pv)*100)],
    ["PV/UV：平均浏览页面数", total_pv/total_uv],
    ["OD/UV：平均每位用的订单数",total_od/total_uv],
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title="天猫双十一战况", subtitle="大盘宏观指标")
)
table.render_notebook()

指标名称,指标数
PV：总浏览页面人数,12256906
UV：独立访客数,10000
OD：总订单数,120205
OD/PV：浏览到购买转化率,0.98%
PV/UV：平均浏览页面数,1225.6906
OD/UV：平均每位用的订单数,12.0205


## 用户生命周期分析

### 用户活跃规律

In [21]:
#流量引入（拉新）：日新增用户及其行为数
da_group=data.groupby(['user_id','date']).count().reset_index()
da_group_drop=da_group.drop_duplicates(subset=['user_id'],keep='first').sort_values('date')

new_user=da_group_drop.groupby('date')['user_id'].count().reset_index().rename(columns={'user_id':'n_user'})
new_pv=da_group_drop.groupby('date')['behavior_type'].sum().reset_index().rename(columns={'behavior_type':'n_pv'})

### 日常新增用户及其行为数

In [22]:
attr = new_user.date.astype(str)
n_pv = new_pv.n_pv
n_user = new_user.n_user

bar = (Bar()
       .add_xaxis(attr.tolist())
       .add_yaxis('日新增UV', n_user.values.tolist(), yaxis_index=0)
       # 加一个Y轴
       .extend_axis(
            yaxis=opts.AxisOpts(
                type_="value",
                position="right",
                axislabel_opts=opts.LabelOpts(formatter="{value}次"))
        )
       .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            min_=0,
            max_=8000,
            interval=2000,
            axislabel_opts=opts.LabelOpts(formatter="{value}人"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="日新增UV和日新增UV的PV的对比图",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
      )

bar.render_notebook()

line = (Line()
       .add_xaxis(attr.values.tolist())
       .add_yaxis('日新增UV的PV', n_pv.values.tolist(),yaxis_index=1,
                 label_opts=opts.LabelOpts(is_show=False))
        
      )

overlap = bar.overlap(line)
overlap.render_notebook()

**在样本时间范围内，日新增用户和日新增用户的浏览量均呈现明显下降趋势。这是由于时间截取，部分老客被认为是新增用户导致的。**

**排除该因素后，日新增用户数量保持在每10000人中有10-30人的区间范围中。回顾淘宝发展历程，这也应证了淘宝在2014年正从野蛮扩展（拉新）逐步走向精细化运营（变现）。**

**同时，在大盘浏览量突升的12月12日，大盘浏览量为691712次。而当日新增用户仅贡献了575次。这说明12月12日活动主要目的是促活促交易，而非拉新**

In [23]:
pv_daily=data.groupby('date')['user_id'].count().rename('pv').reset_index()
uv_daily=data.groupby('date')['user_id'].apply(lambda s:s.drop_duplicates().count()).rename('uv').reset_index()

### 日期维度下的PV和UV

In [24]:
#做出每天的pv与uv趋势图
attr=list(pv_daily.date.astype('str').tolist())
pv=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr)
    .add_yaxis(
        "页面的访问量(PV)",
        np.around(pv_daily.pv/10000,decimals=2),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        series_name="页面的独立访客数(UV)",
        yaxis_index=1,
        y_axis=np.around(uv_daily.uv/1,decimals=2),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="uv",
            type_="value",
            min_=0,
            max_=10000,
            interval=2000,
            axislabel_opts=opts.LabelOpts(formatter="{value} 人"),
        )
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=100,
            interval=20,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万次"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="日期维度下的PV和UV",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="x轴最大", value_index=1),
                opts.MarkPointItem(type_="min", name="x轴最大", value_index=1),
                
            ]))
)

pv.render_notebook()

In [25]:
line = (Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
       .add_xaxis(attr)
       .add_yaxis('',np.around(pv_daily.pv/uv_daily.uv,1))
       .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=100,
            interval=20,
            axislabel_opts=opts.LabelOpts(formatter="{value}次"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="每位独立访问访问数(PV/UV)",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="x轴最大", value_index=1),
                opts.MarkPointItem(type_="min", name="x轴最大", value_index=1),
                
            ]))
      )

line.render_notebook()

**在日期维度下，浏览量（PV）、访客数（UV）和每客浏览量（PV/UV）总体波动趋势大体相同，从8号开始逐步攀升，并在12号当天暴增到达顶峰。其中双十二当天的浏览量、访客数、每客浏览量分别为日常的1.8倍、1.22倍和1.5倍。这都说明双十二活动效果明显，吸引力大，后续的“双十二”的类似活动，可以加长预热期，进一步增强活动的吸引力。**

**双十二当天访客数较日常增长近1500人，说明一些不活跃的用户也被激活，针对此类现象，可结合其他数据重点研究活动的哪种属性刺激了非活跃用户在双十二这天的用户行为，例如优惠，玩法等**

### 小时维度下的PV和UV

In [26]:
#每小时的PV
pv_hour=data.groupby('hour')['user_id'].count().rename('pv').reset_index()
data_pair1 = [(row['hour'],row['pv']) for _, row in pv_hour.iterrows()]
#每小时的UV
uv_hour=data.drop_duplicates(['date','hour','user_id']).groupby('hour')['user_id'].count().rename('uv').reset_index()
data_pair2 = [(row['hour'],row['uv']) for _, row in uv_hour.iterrows()]

In [27]:
line = (Line()
       .add_xaxis([i for i in range(0,24)])
       .add_yaxis('', [x[1]/10000 for x in data_pair1])
      )

line=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis([i for i in range(0,24)])
    .add_yaxis(
        "页面的访问量(PV)",
        np.around([x[1]/10000 for x in data_pair1],decimals=2),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        series_name="页面的独立访客数(UV)",
        yaxis_index=1,
        y_axis=np.around([x[1]/10000 for x in data_pair2],decimals=2),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="uv",
            type_="value",
            min_=0,
            max_=9,
            interval=1.5,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万人"),
        )
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=120,
            interval=20,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万次"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="小时维度下的PV和UV",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="x轴最大", value_index=1),
                opts.MarkPointItem(type_="min", name="x轴最大", value_index=1),
                
            ]))
)

line.render_notebook()

In [28]:
line1 = (Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
       .add_xaxis([i for i in range(0,24)])
       .add_yaxis('',np.around((pv_hour['pv']/uv_hour['uv']).tolist(),1))
       .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=30,
            interval=10,
            axislabel_opts=opts.LabelOpts(formatter="{value}次"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="每位独立访问访问数(PV/UV)",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
    .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="x轴最大", value_index=1),
                opts.MarkPointItem(type_="min", name="x轴最大", value_index=1),
                
            ]))
      )

line1.render_notebook()

**在小时维度下，三项数据都呈现出一定的波动。浏览量（PV）和访客数（UV）在凌晨0-4时期间波动情况相同，都呈下降趋势，访问量都比较小；5-9时期间逐步上升，10-18时相对稳定，19时之后逐步上升，在21时达到峰值后逐步下降。**

**其中，在访客数（UV）相对稳定的情况下，浏览量（PV）在11-12时、17-18时出现小幅波动，这是由于每客浏览量（PV/UV）波动造成的。**

**根据图表数据，10时、13-15时、19-23时是策划运营的三个关键时间段，一些较大型活动设置在用户最为活跃的21-22时期间。**

###  用户留存规律

In [29]:
from datetime import datetime
from datetime import timedelta

def retention(df,n):
    n_date = pd.Series(df.date.unique()).sort_values()[:-n] 
           
    retention_rates = []
    user = []
    for i in n_date:
        new_user = set(df[df.date == i]['user_id'].unique()) - set(user)
        # set() 函数创建一个无序不重复元素集，可进行关系测试，删除重复数据，还可以计算交集、差集、并集等。
        user.extend(new_user)
        n_new_user = df[df.date == i+timedelta(n)]['user_id'].unique()
        a = 0
        for j in n_new_user:
            if j in new_user:
                a+=1
        #print(a,len(new_user))
        retention_rate = a/len(new_user)
        retention_rates.append(retention_rate)
        #print(len(retention_rates),len(n_date))
    total_retention_rate = pd.DataFrame({'date':n_date,'留存率':retention_rates})
    return total_retention_rate

retention_day_1=retention(data,1)
retention_day_3=retention(data,3)
retention_day_5=retention(data,5)
retention_day_7=retention(data,7)

In [30]:
attr2 = retention_day_1.date.astype(str).tolist()
line = (Line()
        .add_xaxis(attr2)
        .add_yaxis('7日留存率', retention_day_7.留存率.tolist())
        .add_yaxis('5日留存率', retention_day_5.留存率.tolist())
        .add_yaxis('3日留存率', retention_day_3.留存率.tolist())
        .add_yaxis('1日留存率', retention_day_1.留存率.tolist())
        .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=False,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="留存率",
            type_="value",
            min_=0,
            max_=1,
            interval=0.2,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="留存率对比分析图",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
        .set_series_opts(
        # 为了不影响标记点，这里把标签关掉
        label_opts=opts.LabelOpts(is_show=False))
      )

line.render_notebook()

**由于数据是一段截取数据，因此前一两天记录的用户可能大多数都是经常上APP的用户，所以留存较高。整体看来，留存率一般维持在40%-45%之间。**

**其中,随着双十二活动的举办，大部分新用户都会在这天上线，所对应的留存率由前期的40%-45%提升至50%左右。**

**而在双十二活动之后，次日留存和3日留存较前期均有所提高，反映出用户粘性在上升。**

### 用户购买情况

In [31]:
# 将数据按照userid进行分组
groupby_userid = data.groupby(by=data.user_id)
# unstack()可以展开再成为一个dataframe
user_type = groupby_userid.behavior_type.value_counts().unstack()

# 跳失率
# user_type.sum(axis=1)对DataFrame进行横向相加，如果一个userid的pv值==横向相加的和，那就表明他只有点击行为，没有购买/加入购物车/收藏的行为
only_pv_users = user_type[user_type['pv'] == user_type.sum(axis=1)]
# shape返回的是元组（行数，列数），所以用[0]取出行数
bounce_rate = only_pv_users.shape[0]/total_uv
# 复购率
user_bought_twice = user_type[user_type['buy'] >= 2].shape[0]
user_bought = user_type[user_type['pv'] >= 1].shape[0]
repurchase_rate = user_bought_twice/user_bought

# print("跳失率:{:.2f}%".format(bounce_rate*100))
# print("复购率:{:.2f}%".format(repurchase_rate*100))
# print(f"总用户数:{total_uv}")

### 复购率与跳失率情况

In [32]:
l1 = (
    Liquid()
    .add("跳失率", [0.034, 0.04], center=["60%", "50%"])
    .set_global_opts(title_opts=opts.TitleOpts(title="复购率与跳失率"),
                    legend_opts=opts.LegendOpts(is_show=True)
    )
)

l2 = Liquid().add(
    "复购率",
    [0.81,0.82],
    center=["25%", "50%"],
    label_opts=opts.LabelOpts(
        font_size=50,
        formatter=JsCode(
            """function (param) {
                    return (Math.floor(param.value * 10000) / 100) + '%';
                }"""
        ),
        position="inside",
    ),
)

grid = Grid().add(l1, grid_opts=opts.GridOpts()).add(l2, grid_opts=opts.GridOpts())
grid.render_notebook()

**用户整体复购率为81%，即81%的用户购买次数超过两次。**

**同时跳失率为3%，用户粘性较好。**


### 用户的购买特征

In [33]:
buy_again_days=data[data.behavior_type=='buy'].groupby(['user_id','date']).count().reset_index()
buy_again_days=buy_again_days.groupby('user_id').date.apply(lambda x:x.sort_values().diff(1).dropna())
buy_again_days=buy_again_days.map(lambda x:x.days).value_counts().reset_index()

In [34]:
line = (Line()
       .add_xaxis([i for i in range(1,31)])
       .add_yaxis('',buy_again_days.date.tolist())
       .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="人数",
            type_="value",
            min_=0,
            max_=15000,
            interval=3000,
            axislabel_opts=opts.LabelOpts(formatter="{value}人"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="用户复购间隔时间及消费次数分布",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=True,pos_top='95%')
    )
    
      )

line.render_notebook()

**图表显示：随着消费时间间隔的增加，有复购行为的用户累积数不断下降。**

**间隔时间在1-10天之内的复购人数比较多，占全部复购人数的93%，10天之后的复购人数很少。**

**因此需要重视10天之内的有购买行为的淘宝用户，采用多元化的相关推荐或者优惠方式，增加用户复购。**

In [35]:
data_user_buy=data[data.behavior_type=='buy'].groupby('user_id')['behavior_type'].count().reset_index().groupby('behavior_type')['user_id'].count().reset_index().rename(columns={'user_id':'num'})
data_user_buy['fenceng']=data_user_buy['behavior_type'].apply(lambda s:(int(s/10)+1)*10)
data_user_buy=data_user_buy.groupby('fenceng')['num'].sum().reset_index()

In [36]:
bar = (Bar()
       .add_xaxis(data_user_buy.fenceng[0:9].tolist())
       .add_yaxis('人数', data_user_buy.num[0:9].tolist(), yaxis_index=0)
    
       .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value}人"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="用户购买次数情况分析",pos_left='center'),
        legend_opts=opts.LegendOpts(is_show=False)
    )
      )

bar.render_notebook()



**55%的淘宝用户消费次数在10次以内，25%的淘宝用户消费次数在10次-20次之间，因此需要重点关注购买次数在20次以上的消费者用户群体。**

## 用户行为路径分析

In [37]:
pv_u_n = data[data.behavior_type=='pv']['user_id'].count()
fav_u_n = data[data.behavior_type == 'fav']['user_id'].count()
cart_u_n = data[data.behavior_type == 'cart']['user_id'].count()
buy_u_n = data[data.behavior_type == 'buy']['user_id'].count()

attr = ['点击', '加入购物车 or 收藏', '购买']

values = [np.around((pv_u_n / pv_u_n * 100), 2),
         np.around(((fav_u_n+cart_u_n)/pv_u_n * 100), 2),
         np.around((buy_u_n / pv_u_n * 100), 2)]
print(values)

[100.0, 5.07, 1.04]


### 转化率分析

In [38]:
funnel = (
    Funnel()
    .add("商品", [list(z) for z in zip(attr, values)])
    .set_global_opts(title_opts=opts.TitleOpts(title="用户购买行为整体转化漏斗"))
)

funnel.render_notebook()

**从浏览到具有购买意向(收藏和加入购物车)，只有5.07%的转化率，但是到了真正到购买的转化率只有1.04%。整体来看，购买转化率最低，有很大的增长空间。
用户流失率最高的是点击—收藏加购这一环节，说明用户的绝大部分行为都是在浏览商品上，寻找符合自己要求的商品，因此需要优化商品的搜索以及推荐等功能，便于用户快速找到合适商品，提高购买转化。**

由于本身的数据集的原因，每条记录的时间最小单位为小时，无法将时间精确到唯一动作。所以这里不对淘宝购物用户的其中的二种路径：(1)点击-未收藏-加购-购买,(2)点击-收藏-加购-购买。不做具体分析的。

接下来对上图，根据漏斗模型对转化率进行分析，流失量大的问题出在第一个环节。接下来就第一个环节转化率低的问题进行进一步的分析与探究。
针对 点击到加入购物车、收藏环节转化率低，用户流失量大的问题，使用**假设检验法**展开深入研究

### 假设检验


![Image Name](https://cdn.kesci.com/upload/image/qny1k34r7e.png?imageView2/0/w/1280/h/1280)


**假设一：商品数量太多，但大部分销量低，无法吸引用户**

收集证据：
(1）先确定商品数量是否足够以及被收藏或加入购物车的商品占比，初步判断是否大部分用品不吸引用户

In [39]:
#商品数
count_itemid = data['item_id'].nunique()
#加入购物车和收藏的总数
item_fav  =  data[data.behavior_type == 'fav']['item_id'].count() #加入收藏的商品
item_cart =  data[data.behavior_type == 'cart']['item_id'].count() #加入购物车的商品
like_item = item_fav + item_cart


cate = ['感兴趣的商品', '不感兴趣的商品']
unlike_item = count_itemid - like_item

num1 = [int(like_item),int(unlike_item)]



pie = (Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
       .add('', [list(z) for z in zip(cate, num1)])
       .set_global_opts(title_opts=opts.TitleOpts(title="用户感兴趣商品占比"))
       .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
       )

pie.render_notebook()

从上图可以看到，用户不敢兴趣大的商品占到的了近80%，这部分商品用户发生了浏览的行为，并在之后迅速流失。

进一步查看的这80%的商品的销售情况。

In [40]:
num_a = data[(data.behavior_type != 'fav') & (data.behavior_type != 'cart')]['item_id'].unique()
B = pd.DataFrame(num_a,columns=['item_id'])
C = pd.merge(B,data,how='left',on='item_id')

In [41]:
num80buy = C[C.behavior_type == 'buy']['item_id'].count()
num80unbuy = len(C)-num80buy 
print(num80unbuy)
cate2 = ['购买', '未购买']
num2 = [int(num80buy),int(num80unbuy)]
print(num2)

12133161
[120205, 12133161]


In [42]:
pie = (Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
       .add('', [list(z) for z in zip(cate2, num2)])
       .set_global_opts(title_opts=opts.TitleOpts(title="未加入购物车的及收藏商品中多少被购买过"))
       .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
       )

pie.render_notebook()

（2）很显然被购买过的商品不足的1%，接下来观察一下这不到2%的商品销量情况

In [43]:
D = C[C.behavior_type == 'buy']['item_id']
E = D.value_counts().sort_values(ascending=False)
itemnums = pd.DataFrame(E)
itemnums['item_id'].value_counts()[:10]

1     77586
2      9910
3      2677
4      1140
5       548
6       346
7       164
8       112
9        68
10       56
Name: item_id, dtype: int64

**结论：假设1成立**

**通过第一步取证得知一共有2876947个商品，商品数量是足够多的，但约80%的商品并不吸引用户。由于销量是影响用户是否愿意购买的一个重要因素，
通过第二步取证得出这80%的商品销量并不可观。只有不到1%的商品被购买过，其中绝大部分商品只被购买过一次。
可以看出没有被加入购物车或收藏的商品太低，不足以吸引用户。
**

##  用户商品偏好分析

### 假设检验二

**假设2：平台推送的商品、商品类目与用户喜好不符**

收集取证：
分别查看点击量前十以及商品购买量前十，对比用户喜欢的与平台推送的吻合度



In [44]:
# 按照itemid进行分组
groupby_itemid = data.groupby(by='item_id')
item_type_df = groupby_itemid.behavior_type.value_counts().unstack()
# 按照categroyid进行分组
groupby_categoryid = data.groupby(by='item_category')
cat_type_df = groupby_categoryid.behavior_type.value_counts().unstack()
# 热卖商品top10
top10_Good = item_type_df.sort_values(by='buy', ascending=False)[:10]


### topN商品

In [45]:
top10_Good 

behavior_type,buy,cart,fav,pv
item_id,,,,
303205878,50.0,46.0,11.0,534.0
14087919,35.0,33.0,15.0,740.0
115124482,31.0,NaN,NaN,20.0
243091690,29.0,3.0,2.0,54.0
167074648,28.0,19.0,4.0,429.0
127914633,24.0,4.0,NaN,29.0
109259240,24.0,14.0,NaN,119.0
101795752,23.0,4.0,NaN,49.0
374214353,23.0,32.0,2.0,320.0


In [46]:
top10_Good = item_type_df.sort_values(by='buy', ascending=False)[:10]
# 点击商品top10
pick10_Good = item_type_df.sort_values(by='pv', ascending=False)[:10]
#类目购买top10
top10_cate = cat_type_df.sort_values(by='fav', ascending=False)[:10]
#推荐类目top10
pick10_cate = cat_type_df.sort_values(by='pv', ascending=False)[:10]


In [47]:
#取证平台推送的点击排名前10的商品与够买前10的商品是否吻合
check_Good = pd.merge(top10_Good,pick10_Good,how='inner',on='item_id')
check_Good

behavior_type,buy_x,cart_x,fav_x,pv_x,buy_y,cart_y,fav_y,pv_y
item_id,,,,,,,,
14087919,35.0,33.0,15.0,740.0,35.0,33.0,15.0,740.0


In [48]:
#取证平台推送的购买前10的类目与点击量前10类目是否吻合
check_cate = pd.merge(top10_cate,pick10_cate,how='inner',on='item_category')
check_cate

behavior_type,buy_x,cart_x,fav_x,pv_x,buy_y,cart_y,fav_y,pv_y
item_category,,,,,,,,
1863,2000.0,9309.0,10200.0,371738.0,2000.0,9309.0,10200.0,371738.0
5894,958.0,6615.0,7850.0,314784.0,958.0,6615.0,7850.0,314784.0
13230,841.0,6012.0,7226.0,342694.0,841.0,6012.0,7226.0,342694.0
5027,858.0,5564.0,6988.0,320870.0,858.0,5564.0,6988.0,320870.0
6513,1059.0,6651.0,6688.0,281370.0,1059.0,6651.0,6688.0,281370.0
5399,1054.0,5430.0,6616.0,268639.0,1054.0,5430.0,6616.0,268639.0
11279,722.0,3686.0,4687.0,177961.0,722.0,3686.0,4687.0,177961.0
2825,625.0,3692.0,3360.0,155949.0,625.0,3692.0,3360.0,155949.0


**结论：假设2成立**

**数据显示，点击量前十的商品类目与购买量前十的商品类目符合度为80%，比较良好。而进一步对商品进行分析发现平台推送的商品与用户喜欢购买的前十商品中符合度为10%，能看出平台缺少畅销品和爆款。说明平台推送的商品针对性不强，导致点击浏览商品详情后，用户流失量大。**

## 用户价值分析

### RFM模型

In [49]:
last_time=data[data.behavior_type=='buy'].groupby('user_id').max()['date']
recency=(pd.to_datetime('2014-12-18')-last_time).dt.days.copy() # 这里创建副本是为了防止下面用索引设置R值的时候出现链式赋值警告
recency_avg=recency.mean()
recency[recency<recency_avg]=0
recency[recency>recency_avg]=1

In [50]:
# 构建F值
frequency=data[data.behavior_type=='buy'].groupby('user_id').count()['item_id'].copy()
frq_avg=frequency.mean()
frequency[frequency<frq_avg]=0
frequency[frequency>frq_avg]=1

In [51]:
rfm=pd.merge(recency,frequency,on='user_id',how='inner')
rfm=rfm.reset_index().rename(columns={'date':'r','item_id':'f'})
rfm=rfm[['r','f']].astype('str')
rfm['user_type']=rfm['r']+rfm['f']
RFM = rfm.groupby('user_type').count()['r'].sort_index(ascending=False).rename(index={'11':'重要价值用户：','10':'重要发展用户：','01':'重要保持用户：','00':'重要挽留用户：'})


In [52]:
RFM = RFM.sort_values( ascending=False)
RFM

user_type
重要发展用户：    3055
重要挽留用户：    2986
重要保持用户：    2394
重要价值用户：     451
Name: r, dtype: int64

In [53]:
data_pair = [('重要价值客户', 451), ('重要发展客户', 3055), ('重要保持客户', 2394), ('重要挽留客户', 2986)]
pie = (Pie(init_opts=opts.InitOpts(theme='dark', width='30px',height='50px'))
       .add('', data_pair,
            radius=['20%','40%'],center=['72%','25%'],
            )
       .set_global_opts(title_opts=opts.TitleOpts(title="客户群体分析",
                                                  subtitle='RFM模型'),
                        legend_opts=opts.LegendOpts(is_show=False),)
       .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
      )


bar = (Bar(init_opts=opts.InitOpts(theme='dark'))
       .add_xaxis(RFM.index.tolist())
       .add_yaxis('分层群体', RFM.values.tolist(),bar_width='40%')
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True, font_style='italic'),
                            itemstyle_opts=opts.ItemStyleOpts(
                                color=JsCode("""new echarts.graphic.LinearGradient(0, 1, 0, 0, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(32,178,170)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(255,99,71)'
                                             }])"""))
                            )
       .set_global_opts(
           title_opts=opts.TitleOpts(title="客户群体分析",subtitle='LRFMC模型'),
           legend_opts=opts.LegendOpts(is_show=False),
           tooltip_opts=opts.TooltipOpts(formatter='{b}:{c}人'))
      )

bar.overlap(pie)
bar.render_notebook()


**重点价值用户是最重要的用户群体，但数量较少，只有451人，而其余类型的用户群体数量相当，均有2000人以上的规模，需针对用户群体特点针对性采取运营措施。
**

**重要价值用户是最优质的用户群体，应重点关注，既要保持其粘性，又要继续引导消费，可为这类用户提供vip服务；**

**重要发展用户的特点是近期有消费但频次不高，策略是提高其消费次数，具体措施有促销活动提醒和优惠卷活动等；**

**重要保持用户的特点是消费频次高但有一段时间没有消费，策略是重新唤醒，通过app消息推送，以及站外广告营销吸引其注意力，促进复购**

**重要挽留用户近期没有消费且频次不高，若不加以挽留，会有流失的可能，对于这类用户一方面需要保持曝光量，持续推送活动和优惠信息，另一方面需要进一步研究其兴趣和需求，才能采取有效的运营策略**

# 整体结论与策略落地

##  1.淘宝用户的基本情况

本次数据集中有大部分是是付费用户，并且这一个月内的复购率也达到了81%，说明淘宝已经深入我们的日常生活了，付费转化率非常高，所以，我们只需要培养用户的忠诚度，让用户继续在淘宝上消费，并且通过一些营销政策，优惠策略等提高用户的消费频次。

尤其是现在东京占领高端头部市场，同时出现了拼多多这样一个下沉到农村市场的电商平台，占领长尾市场。淘宝更应该抓住那部分对价格敏感的用户，做一些转发拼团，发放优惠券等的活动，来增加用户。

## 2.在时间维度上

在用户的休闲时间段，比如通勤时间7-10点，午饭时间12-13点，以及下班后晚上8-10点之间做促销活动，以及一些相关的营销，
多招聘像**李佳奇，薇娅**等人才或者流量明星。，在活跃时段进行直播带货。打造爆款明星产品。这样能够大大增加用户购买的转化率。

## 3.业务流程方面

通过一些方法，引导用户进行收藏和加入购物车，比如发放优惠券，满减活动，送礼品等给用户优惠的方式，通过大量销售，提升购买转化率。并且对于点击量高的商品，要重点分析。技术上进行大数据数据挖掘，优化商品的推荐机制，千人千面。让用户做到点击即想购买。


## 4.用户群体

对于重要价值的用户，需要给其提供VIP服务，比如VIP可以得到飞猪优惠券，天猫超市优惠券，网易云，饿了么联名卡等等

![Image Name](https://cdn.kesci.com/upload/image/qnzojbsjg0.png?imageView2/0/w/960/h/960)

对于重要挽留的客户，要发短信和邮件，或者APP客户端推送等，让客户重新回来消费，此外可以通过一些节日优惠等再次召回客户。

对于重要发展和重要保持用户，可以再重点提升消费频次，通过优惠券等活动刺激消费。


## 5.商品推荐

通过将畅销商品和非畅销商品进行捆绑销售，优化商品的展示，建立高频网红专区。
当然在做这些活动之前，可以先尝试做 A/B 测试，找到最适合的销售方式和最合适的商品类型捆绑，不断提升转化率。

end